In [5]:

import torch
from torch import nn
from transformers import XLMRobertaModel, XLMRobertaTokenizer
from torch.utils.data import Dataset, DataLoader
from typing import List, Dict, Optional
import numpy as np

In [6]:
import pandas as pd

In [ ]:
train_df = pd.read_csv("combined_lang_train.csv")

In [13]:
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [15]:
train_df.shape

(1781, 98)

In [16]:
train_df

,file,text,CC: Amplifying Climate Fears: Amplifying existing fears of global warming,CC: Amplifying Climate Fears: Doomsday scenarios for humans,CC: Amplifying Climate Fears: Earth will be uninhabitable soon,CC: Amplifying Climate Fears: Whatever we do it is already too late,CC: Amplifying Climate Fears: Other,CC: Climate change is beneficial: CO2 is beneficial,CC: Climate change is beneficial: Temperature increase is beneficial,CC: Climate change is beneficial: Other,...,URW: Praise of Russia: Other,URW: Russia is the Victim: Russia actions in Ukraine are only self-defence,URW: Russia is the Victim: The West is russophobic,URW: Russia is the Victim: UA is anti-RU extremists,URW: Russia is the Victim: Other,URW: Speculating war outcomes: Russian army is collapsing,URW: Speculating war outcomes: Russian army will lose all the occupied territories,URW: Speculating war outcomes: Ukrainian army is collapsing,URW: Speculating war outcomes: Other,Other
0,HI_128.txt,पुतिन ने कमला हैरिस के बारे में मुस्कुराकर क्य...,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,PT_40.txt,Sobre os ataques de Kiev às estações de radar ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,EN_UA_023816.txt,NATO Troops Are Operating In Ukraine Disguised...,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
3,BG_939.txt,ВТЪЛПЯВАНЕТО НА ВИНА ОБСЛУЖВА ГЛОБАЛИСТИТЕ\n\n...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BG_704.txt,"Всеки път, размишлявайки върху териториалната ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1776,HI_62.txt,"यूक्रेन ने निंदा की कि रूस डोनेट्स्क, लुहान्स्...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1777,PT_222.txt,Itália prepara-se para vaga de calor recorde c...,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1778,HI_171.txt,"अब रूस-यूक्रेन युद्ध में मच सकता है कोहराम, US...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1779,PT_39.txt,"""É necessário usar Zelensky"": um apelo ao ""car...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:

class MultilingualMultiLabelClassifier(nn.Module):
    def __init__(self, num_classes: int = 96, dropout: float = 0.1):
        super().__init__()
        
        # Load pre-trained XLM-RoBERTa model
        self.roberta = XLMRobertaModel.from_pretrained('xlm-roberta-base')
        
        # Hidden dimensions
        self.hidden_dim = self.roberta.config.hidden_size
        
        # Multi-label classification head
        self.classifier = nn.Sequential(
            nn.Linear(self.hidden_dim, self.hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(self.hidden_dim, num_classes),
            nn.Sigmoid()  # Sigmoid for multi-label classification
        )

    def forward(self, input_ids, attention_mask):
        # Get RoBERTa embeddings
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        
        # Use [CLS] token representation for classification
        pooled_output = outputs.last_hidden_state[:, 0, :]
        
        # Pass through classification head
        logits = self.classifier(pooled_output)
        
        return logits

In [8]:


class MultilingualMultiLabelDataset(Dataset):
    def __init__(self, texts: List[str], labels: Optional[np.ndarray] = None, 
                 max_length: int = 512):
        """
        Args:
            texts: List of input texts
            labels: numpy array of one-hot encoded labels (num_samples x num_classes)
            max_length: Maximum sequence length
        """
        self.texts = texts
        self.labels = labels
        self.max_length = max_length
        self.tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        
        # Tokenize text
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        item = {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
        }

        if self.labels is not None:
            item['labels'] = torch.FloatTensor(self.labels[idx])  # Convert to float for BCELoss

        return item

In [23]:
from tqdm.auto import tqdm

In [ ]:
def train_model(model, train_dataloader, val_dataloader, 
                num_epochs=5, learning_rate=2e-5, device='cuda:1'):
    # Move model to device
    model = model.to(device)
    
    # Initialize optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    
    # Binary Cross Entropy Loss for multi-label classification
    criterion = nn.BCELoss()
    
    # Training loop
    progress_bar = tqdm(range(num_epochs), desc="Training", position=0)
    for epoch in progress_bar:
        model.train()
        total_loss = 0
        
        # Create progress bar for batches
        batch_progress = tqdm(train_dataloader, 
                            desc=f"Epoch {epoch+1}/{num_epochs}",
                            leave=False,
                            position=1)
        
        for batch in batch_progress:
            # Move batch to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            # Forward pass
            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            
            # Calculate loss
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            # Update batch progress bar
            batch_progress.set_postfix({"loss": f"{loss.item():.4f}"})
        
        # Validation
        model.eval()
        val_loss = 0
        all_predictions = []
        all_labels = []
        
        with torch.no_grad():
            val_progress = tqdm(val_dataloader, 
                              desc="Validation",
                              leave=False,
                              position=1)
            for batch in val_progress:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                # Convert predictions to binary (0 or 1) using threshold of 0.5
                predictions = (outputs > 0.5).float()
                
                all_predictions.extend(predictions.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        # Calculate metrics
        all_predictions = np.array(all_predictions)
        all_labels = np.array(all_labels)
        
        # Calculate accuracy, precision, recall, f1 per class
        accuracy = np.mean(all_predictions == all_labels)
        
        # Calculate hamming loss (fraction of wrong labels)
        hamming_loss = np.mean(all_predictions != all_labels)
        
        # Update progress bar with metrics
        metrics = {
            "train_loss": f"{total_loss/len(train_dataloader):.4f}",
            "val_loss": f"{val_loss/len(val_dataloader):.4f}",
            "accuracy": f"{accuracy:.4f}",
            "hamming_loss": f"{hamming_loss:.4f}"
        }
        progress_bar.set_postfix(metrics)
        
        tqdm.write(f"\nEpoch {epoch+1}/{num_epochs}:")
        tqdm.write(f"Training Loss: {metrics['train_loss']}")
        tqdm.write(f"Validation Loss: {metrics['val_loss']}")
        tqdm.write(f"Accuracy: {metrics['accuracy']}")
        tqdm.write(f"Hamming Loss: {metrics['hamming_loss']}\n")

In [3]:

def predict(model, text: str, threshold: float = 0.5, device='cuda:1'):
    model.eval()
    tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
    
    # Tokenize input text
    encoding = tokenizer(
        text,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    # Move to device
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # Get prediction
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probabilities = outputs.squeeze()
        predicted_labels = (probabilities > threshold).cpu().numpy()
        class_probabilities = probabilities.cpu().numpy()
    
    return predicted_labels, class_probabilities

# Example usage

In [ ]:

# Initialize model
model = MultilingualMultiLabelClassifier(num_classes=96)

# Sample data (replace with your actual data)
# texts = ["Sample text in any language"] * 100
texts = train_df["text"].values
# Create random one-hot encoded labels for demonstration
labels = train_df.iloc[:, 2:].values
# labels = np.random.randint(0, 2, size=(100, 96)).astype(np.float32)

# Create dataset
dataset = MultilingualMultiLabelDataset(texts=texts, labels=labels)

# Create data loaders
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16)

# Train model
train_model(model, train_dataloader, val_dataloader,num_epochs=100)


In [33]:

text = "Ukraine is responsible for the war. It's closeness to NATO is the reason for the war"
predicted_labels, class_probabilities = predict(model, text)
print("Predicted labels (1 indicates presence of class):")
print(predicted_labels)
print("\nClass probabilities:")
print(class_probabilities)

Predicted labels (1 indicates presence of class):
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False  True False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False]

Class probabilities:
[6.05424646e-07 4.84351722e-06 1.64131825e-06 2.19881613e-07
 5.09494498e-07 2.74752328e-08 1.04448702e-07 1.13007470e-08
 6.05802630e-11 3.45098101e-08 3.63955728e-08 1.57961480e-07
 4.34269936e-07 5.13727088e-08 1.18929480e-08 1.99340420e-08
 1.39503868e-07 7.44456258e-07 6.05109960e-07 5.76414095e-07
 4.25779831e-07 5.95227334e-08 3.72625

In [20]:
predicted_labels.sum()

np.int64(1)

In [ ]:
dev_df = pd.read_csv("../task_data/subtask2_hf/EN_fine-grained/dev.csv")

In [35]:

texts = dev_df["text"].values
all_predictions = []
for text in texts:
    predicted_labels, class_probabilities = predict(model, text)
    # print("Predicted labels (1 indicates presence of class)?:")

    all_predictions.append(np.array(predicted_labels).astype(int))
    # print("\nClass probabilities:")

In [36]:
res_df = dev_df[["file","text"]]

In [37]:
res_df = pd.concat([res_df,pd.DataFrame(all_predictions,columns=dev_df.columns[2:])],axis=1)

In [38]:
res_df.to_csv("results_finegrained.csv",index=False)

In [39]:
dev_df.iloc[:,2:].iloc[0] == predicted_labels.sum


CC: Amplifying Climate Fears: Amplifying existing fears of global warming             False
CC: Amplifying Climate Fears: Doomsday scenarios for humans                           False
CC: Amplifying Climate Fears: Earth will be uninhabitable soon                        False
CC: Amplifying Climate Fears: Whatever we do it is already too late                   False
CC: Amplifying Climate Fears: Other                                                   False
                                                                                      ...  
URW: Speculating war outcomes: Russian army is collapsing                             False
URW: Speculating war outcomes: Russian army will lose all the occupied territories    False
URW: Speculating war outcomes: Ukrainian army is collapsing                           False
URW: Speculating war outcomes: Other                                                  False
Other                                                                           

In [40]:

tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

In [41]:
save_model(model, tokenizer, "./model")

Model and tokenizer saved to ./model


In [2]:
def save_model(model, tokenizer, output_dir: str):
    """
    Save the model and tokenizer to disk
    
    Args:
        model: The trained model
        tokenizer: The tokenizer
        output_dir: Directory to save the model and tokenizer
    """
    import os
    import json
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Save the base RoBERTa model
    model.roberta.save_pretrained(os.path.join(output_dir, "roberta_base"))
    
    # Save tokenizer
    tokenizer.save_pretrained(output_dir)
    
    # Save model state dict (includes classifier layers)
    torch.save(model.state_dict(), os.path.join(output_dir, "model_state_dict.pt"))
    
    # Save model configuration
    config = {
        'num_classes': 96,
        'hidden_dim': model.hidden_dim,
        'dropout': 0.1  # Save dropout rate if needed
    }
    
    with open(os.path.join(output_dir, "model_config.json"), 'w') as f:
        json.dump(config, f)
    
    print(f"Model and tokenizer saved to {output_dir}")

def load_model(model_dir: str, device='cuda'):
    """
    Load the model and tokenizer from disk
    
    Args:
        model_dir: Directory containing the saved model and tokenizer
        device: Device to load the model on
    
    Returns:
        model: The loaded model
        tokenizer: The loaded tokenizer
    """
    import os
    import json
    
    # Load model configuration
    config_path = os.path.join(model_dir, "model_config.json")
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Model config not found at {config_path}")
    
    with open(config_path, 'r') as f:
        config = json.load(f)
    
    # Initialize model with saved configuration
    model = MultilingualMultiLabelClassifier(
        num_classes=config['num_classes']
    )
    
    # Load the base RoBERTa model
    roberta_path = os.path.join(model_dir, "roberta_base")
    model.roberta = XLMRobertaModel.from_pretrained(roberta_path)
    
    # Load tokenizer
    tokenizer = XLMRobertaTokenizer.from_pretrained(model_dir)
    
    # Load state dict
    state_dict_path = os.path.join(model_dir, "model_state_dict.pt")
    if os.path.exists(state_dict_path):
        model.load_state_dict(torch.load(state_dict_path, map_location=device))
    else:
        raise FileNotFoundError(f"Model state dict not found at {state_dict_path}")
    
    model = model.to(device)
    model.eval()
    
    return model, tokenizer


# Prepare results for testing

In [2]:
import pandas as pd

In [36]:
res_df = pd.read_csv("results_finegrained.csv")

In [37]:
# Function to get labels for each row
def get_labels(row):
    # Skip the 'file' and 'text' columns
    label_columns = row.index[2:]
    # Get column names where value is 1
    labels = [col for col in label_columns if row[col] == 1]
    # Clean up the labels (remove the category prefix if desired)
    labels = [label.split(': ')[-1] for label in labels]
    return labels

# Create a new column with the labels
res_df['labels'] = res_df.apply(get_labels, axis=1)

# Keep only essential columns
res_df = res_df[['file', 'text', 'labels']]

In [38]:
res_df["len"] = res_df["labels"].apply(lambda x: len(x))

In [39]:
res_df

,file,text,labels,len
0,EN_CC_200030.txt,"Bangladesh, Nordic countries to strengthen coo...",[Other],1
1,EN_CC_200033.txt,Greta Thunberg Calls For 'Overthrow of Whole C...,[Ad hominem attacks on key activists],1
2,EN_CC_200034.txt,If we “just stop oil” like climate protesters ...,[Climate agenda has hidden motives],1
3,EN_CC_200035.txt,Gretchen Whitmer Orders Michigan’s State Fleet...,[Other],1
4,EN_CC_200036.txt,Climate cultists push bizarre scare language a...,[Ad hominem attacks on key activists],1
5,EN_CC_200040.txt,Climate Protesters Out Of Control As They Atta...,[],0
6,EN_CC_200046.txt,CO2 is the GREENING molecule: New study shows ...,[],0
7,EN_CC_200047.txt,89% of ‘American Elites’ Back WEF’s Plan to Ra...,"[Other, Climate movement is alarmist, Criticis...",6
8,EN_CC_200049.txt,Alarmists Warn of U.S. ‘Heat Dome’ Tied to Hum...,[],0
9,EN_CC_200050.txt,Link to Major Banks Bend The Knee To Climate A...,[],0


In [40]:
res_df['label_fin'] = res_df['labels'].apply(lambda x: ';'.join(x) if len(x) > 0 else '')

In [44]:
res_df['label_fin'] = res_df.apply(lambda x: 'Other' if x['len'] == 0 else x['label_fin'], axis=1)

In [48]:
res_df = res_df[["file", "label_fin"]]

In [49]:
res_df

,file,label_fin
0,EN_CC_200030.txt,Other
1,EN_CC_200033.txt,Ad hominem attacks on key activists
2,EN_CC_200034.txt,Climate agenda has hidden motives
3,EN_CC_200035.txt,Other
4,EN_CC_200036.txt,Ad hominem attacks on key activists
5,EN_CC_200040.txt,Other
6,EN_CC_200046.txt,Other
7,EN_CC_200047.txt,Other;Climate movement is alarmist;Criticism o...
8,EN_CC_200049.txt,Other
9,EN_CC_200050.txt,Other


# Load model for multilingual

In [32]:
model = load_model("./model", device='cuda:1')[0]

/tmp/ipykernel_1092956/756521338.py:75: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(state_dict_path, map_location=device))


In [33]:
model

MultilingualMultiLabelClassifier(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768

In [34]:
text = "I am a quack and climate change is fraud"

In [35]:
predicted_labels, class_probabilities = predict(model, text) 

In [36]:
predicted_labels

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])

In [ ]:
en_dev_df = pd.read_csv("../task_data/subtask2_hf/EN_fine-grained/dev.csv")
bg_dev_df = pd.read_csv("../task_data/subtask2_hf/BG_fine-grained/dev.csv")
hi_dev_df = pd.read_csv("../task_data/subtask2_hf/HI_fine-grained/dev.csv")
pt_dev_df = pd.read_csv("../task_data/subtask2_hf/PT_fine-grained/dev.csv")
ru_dev_df = pd.read_csv("../task_data/subtask2_hf/RU_fine-grained/dev.csv")


# Predict for each language

In [38]:

texts = en_dev_df["text"].values
all_predictions = []
for text in texts:
    predicted_labels, class_probabilities = predict(model, text)
    # print("Predicted labels (1 indicates presence of class)?:")

    all_predictions.append(np.array(predicted_labels).astype(int))

res_df = en_dev_df[["file","text"]]
res_df = pd.concat([res_df,pd.DataFrame(all_predictions,columns=en_dev_df.columns[2:])],axis=1)
res_df.to_csv("en_results_onehot.csv",index=False)

In [45]:
from tqdm.auto import tqdm

In [ ]:
import pandas as pd
import numpy as np

# List of language codes
languages = ['EN', 'RU', 'PT', 'BG', 'HI']

# Process each language
for lang in languages:
    # Load the dataframe
    df = pd.read_csv(f"../task_data/subtask2_hf/{lang}_fine-grained/dev.csv")
    # df = pd.read_csv(f"{lang}_dev_df.csv")
    
    # Get texts
    texts = df["text"].values
    all_predictions = []
    
    # Process each text with progress bar
    for text in tqdm(texts, desc=f"Processing {lang.upper()}"):
        predicted_labels, _ = predict(model, text)
        all_predictions.append(np.array(predicted_labels).astype(int))
    
    # Create results dataframe
    res_df = df[["file", "text"]]
    res_df = pd.concat([res_df, pd.DataFrame(all_predictions, columns=df.columns[2:])], axis=1)
    
    # Save results
    res_df.to_csv(f"{lang}_results_onehot.csv", index=False)
    print(f"Saved {lang}_results_onehot.csv")

Processing EN:   0%|          | 0/41 [00:00<?, ?it/s]

Saved EN_results_onehot.csv


Processing RU:   0%|          | 0/32 [00:00<?, ?it/s]

Saved RU_results_onehot.csv


Processing PT:   0%|          | 0/35 [00:00<?, ?it/s]

Saved PT_results_onehot.csv


Processing BG:   0%|          | 0/35 [00:00<?, ?it/s]

Saved BG_results_onehot.csv


Processing HI:   0%|          | 0/35 [00:00<?, ?it/s]

Saved HI_results_onehot.csv


In [47]:
del model